## Задание

С помощью базы данных StackOverflow — сервиса вопросов и ответов о программировании - сделать различные выгрузки

In [ ]:
# задание 1
# Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

SELECT COUNT(*)
FROM stackoverflow.posts 
WHERE post_type_id = 1
AND (score > 300 
OR favorites_count >= 100)
 

In [ ]:
# задание 2
# Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

WITH selection AS (
    SELECT creation_date::date AS dt,
           COUNT(id) AS cnt
FROM stackoverflow.posts
WHERE creation_date::date BETWEEN '2008-11-01' AND '2008-11-18'
AND post_type_id = 1    
GROUP BY 1)

SELECT ROUND(AVG(cnt))
FROM selection


In [ ]:
# задание 3
# Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

SELECT COUNT( DISTINCT b.user_id)
FROM stackoverflow.badges AS b
JOIN stackoverflow.users AS u ON u.id = b.user_id
WHERE b.creation_date::date = u.creation_date::date

In [ ]:
# задание 4
# Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.votes AS v ON p.id = v.post_id
WHERE p.user_id IN (
SELECT id
FROM stackoverflow.users
WHERE display_name = 'Joel Coehoorn')

In [ ]:
# задание 5
# Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

SELECT *,
       RANK() OVER (ORDER BY id DESC)
FROM stackoverflow.vote_types
ORDER BY 1

In [ ]:
# задание 6
# Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

SELECT DISTINCT user_id,
       COUNT(id) 
FROM stackoverflow.votes
WHERE vote_type_id IN (SELECT id
FROM stackoverflow.vote_types
WHERE name = 'Close')
GROUP BY 1
ORDER BY 2 DESC,
         1 DESC
LIMIT 10 

In [ ]:
# задание 7
# Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно. Отобразите несколько полей:
# *идентификатор пользователя;
# *число значков;
# *место в рейтинге — чем больше значков, тем выше рейтинг.
#Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
#Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

WITH selection AS (SELECT DISTINCT user_id,
       COUNT(id) OVER (PARTITION BY user_id) AS cnt
FROM stackoverflow.badges
WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
ORDER BY 2 DESC,
         1
LIMIT 10)
  
SELECT *,
       DENSE_RANK() OVER (ORDER BY cnt DESC)
FROM selection

In [ ]:
# задание 8
# Сколько в среднем очков получает пост каждого пользователя? Сформируйте таблицу из следующих полей: заголовок поста; идентификатор пользователя; число очков поста; среднее число очков пользователя за пост, округлённое до целого числа.
#Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY user_id))
FROM stackoverflow.posts
WHERE title IS NOT NULL
AND score != 0

In [ ]:
# задание 9
# Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

SELECT title
FROM stackoverflow.posts
WHERE user_id IN
    (SELECT user_id
    FROM stackoverflow.badges
    GROUP BY 1
    HAVING COUNT(id) > 1000)
AND title IS NOT NULL 


In [ ]:
# задание 10
# Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
# пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
# пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
# пользователям с числом просмотров меньше 100 — группу 3.
# Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

SELECT id,
       views,
       CASE
           WHEN views >= 350 THEN 1
           WHEN views < 100 THEN 3
           ELSE 2
       END
FROM stackoverflow.users
WHERE location LIKE '%United States%' AND views != 0

In [ ]:
# задание 11
# Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

WITH selection AS (SELECT id,
       views,
       CASE
           WHEN views >= 350 THEN 1
           WHEN views < 100 THEN 3
           ELSE 2
       END AS group_123
FROM stackoverflow.users
WHERE location LIKE '%United States%' AND views != 0),
max_by_group AS 
    (SELECT group_123,
            MAX(views)
     FROM selection
     GROUP BY 1) 

SELECT selection.id,
       max_by_group.group_123,
       max_by_group.max
FROM selection 
JOIN max_by_group ON selection.views = max_by_group.max
ORDER BY 3 DESC,
         1 

In [ ]:
# задание 12
# Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями: номер дня; число пользователей, зарегистрированных в этот день; сумму пользователей с накоплением.

WITH selection AS (SELECT DISTINCT EXTRACT(DAY FROM creation_date::date) dt,
                          COUNT(id) OVER (PARTITION BY EXTRACT(DAY FROM creation_date::date)) AS cnt
                   FROM stackoverflow.users
                   WHERE EXTRACT(MONTH FROM creation_date::date) = 11
                   AND EXTRACT(YEAR FROM creation_date::date) = 2008
                  )

SELECT *,
       SUM(cnt) OVER (ORDER BY dt) 
FROM selection


In [ ]:
# задание 13
#  Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите: идентификатор пользователя; разницу во времени между регистрацией и первым постом.

WITH date_first_post AS (SELECT user_id,
                                MIN(creation_date) dt
                         FROM stackoverflow.posts
                         GROUP BY 1
                        )

SELECT date_first_post.user_id,
       date_first_post.dt - u.creation_date
FROM date_first_post
JOIN stackoverflow.users AS u ON u.id = date_first_post.user_id

In [ ]:
#задание 14
# Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

SELECT DISTINCT DATE_TRUNC('month', creation_date)::date dt,
       SUM(views_count) OVER (PARTITION BY DATE_TRUNC('month', creation_date)::date) 
FROM stackoverflow.posts 
ORDER BY 2 DESC


In [1]:
#задание 15
#Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

SELECT u.display_name,
       COUNT(DISTINCT p.user_id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON u.id = p.user_id
WHERE p.post_type_id = 2 
AND DATE_TRUNC('day', p.creation_date) <= DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month'
AND DATE_TRUNC('day', p.creation_date) >= DATE_TRUNC('day', u.creation_date)
GROUP BY 1
HAVING COUNT(p.id) > 100     
ORDER BY 1

In [ ]:
# задание 16
#Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

SELECT DATE_TRUNC('month', creation_date)::date,
       COUNT(*) 
FROM stackoverflow.posts
WHERE user_id IN 
(SELECT DISTINCT u.id
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p ON u.id = p.user_id
WHERE DATE_TRUNC('month', u.creation_date)::date = '2008-09-01'
AND DATE_TRUNC('month', p.creation_date)::date = '2008-12-01')
GROUP BY 1
ORDER BY 1 DESC


In [ ]:
#задание 17
# Используя данные о постах, выведите несколько полей: идентификатор пользователя, который написал пост; дата создания поста; количество просмотров у текущего поста; сумму просмотров постов автора с накоплением.
# Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date) 
FROM stackoverflow.posts
ORDER BY 1

In [ ]:
#задание 18
# Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

WITH selection AS (SELECT user_id,
       COUNT(DISTINCT creation_date::date) AS cnt
FROM stackoverflow.posts
WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07'
GROUP BY 1)

SELECT ROUND(AVG(cnt))
FROM selection

In [ ]:
#задание 19
# На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
# номер месяца;
# количество постов за месяц;
# процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
# Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

WITH selection AS 
(SELECT EXTRACT(month FROM creation_date) AS dt,
       COUNT(id) AS cnt
FROM stackoverflow.posts
WHERE EXTRACT(month FROM creation_date) IN (9, 10, 11, 12) 
GROUP BY 1
)

SELECT *,
       ROUND((cnt::numeric/LAG(cnt) OVER (ORDER BY dt)-1)*100,2)
FROM selection    

In [ ]:
#задание 20
#Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
# номер недели;
# дата и время последнего поста, опубликованного на этой неделе.

WITH selection AS (SELECT EXTRACT('week' FROM creation_date) AS week,
       creation_date 
FROM stackoverflow.posts
WHERE user_id in (SELECT user_id
FROM stackoverflow.posts
GROUP BY 1
ORDER BY COUNT(id) DESC
LIMIT 1)
AND DATE_TRUNC('month', creation_date)::date = '2008-10-01'
ORDER BY 1,
         2)

SELECT DISTINCT week,
       LAST_VALUE(creation_date) OVER (PARTITION BY week ORDER BY EXTRACT('week' FROM creation_date)) 
FROM selection 